In [1]:
import h5py
import numpy as np
import pickle as pk
import os
from scipy.stats import gmean

In [2]:
#hdf5=h5py.File('/project/projectdirs/polar/data/largepatch_reanalysis/largepatch_planck_sim_set/lowell_2_20210220/final_spectra_pack.hdf5','r')
hdf5=h5py.File('/project/projectdirs/polar/data/largepatch_reanalysis/largepatch_planck_sim_set/lowell_4_20210328/final_spectra_pack_unblind.hdf5','r' )

In [3]:
spectra_keys = ['tt', 'ee', 'bb', 'te', 'tb', 'eb']
freq_name = ['planck_143','pb', 'planck_217','planck_353']

In [4]:
def dump_noise(hdf5,name):
    name_ = 'polarbear' if name=='pb' else name

    foldername = os.path.join('polar/noise_bias',name)
    os.makedirs(foldername,exist_ok=True)
    noise_bias = {}
    for keys in spectra_keys:
        noise_bias[keys] = {}
        noise_bias[keys]['Cb'] = hdf5[f"noise_fullcross/{name_}x{name_}/{keys}"]['Cb'][:,:].mean(axis=0)
    filename = os.path.join(foldername,'allspec.pkl')

    pk.dump(noise_bias, open(filename, 'wb'))

In [12]:
for name in freq_name:
    dump_noise(hdf5,name)

In [5]:
def make_filename(exp1,exp2):
    if exp1 == exp2:
        return exp1
    else:
        if exp1 == "pb":
            return f"{exp2}_{exp1}"
        else:
            return f"{exp1}_{exp2}"

def make_keyname(exp1,exp2):
    exp1 = 'polarbear' if exp1 == 'pb' else exp1
    exp2 = 'polarbear' if exp2 == 'pb' else exp2
    
    if exp1 == 'polarbear':
        return f"{exp2}x{exp1}"
    else:
        return f"{exp1}x{exp2}"
      

In [6]:
def dump_spectra(hdf5,exp1,exp2):
    name = make_filename(exp1,exp2)
    foldername = os.path.join('polar/real_spectra_abscal_pb_fakeauto/',name)
    os.makedirs(foldername,exist_ok=True)
    spectra = {}
    spectra['bins'] = hdf5['signal_fullcross']['bins'][:].flatten()
    for keys in spectra_keys:
        spectra[keys] = {}
        key = f"real_fullcross/{make_keyname(exp1,exp2)}/{keys}"
        spectra[keys]['Cb'] = hdf5[key]['Cb'][:]
    filename = os.path.join(foldername,'allspec.pkl')
    print(f"writing {filename}")
    pk.dump(spectra,open(filename,'wb'))

In [15]:
for i, exp1 in enumerate(freq_name):
    for j, exp2 in enumerate(freq_name):
        if i <= j:
            dump_spectra(hdf5,exp1,exp2)

writing polar/real_spectra_abscal_pb_fakeauto/planck_143/allspec.pkl
writing polar/real_spectra_abscal_pb_fakeauto/planck_143_pb/allspec.pkl
writing polar/real_spectra_abscal_pb_fakeauto/planck_143_planck_217/allspec.pkl
writing polar/real_spectra_abscal_pb_fakeauto/planck_143_planck_353/allspec.pkl
writing polar/real_spectra_abscal_pb_fakeauto/pb/allspec.pkl
writing polar/real_spectra_abscal_pb_fakeauto/planck_217_pb/allspec.pkl
writing polar/real_spectra_abscal_pb_fakeauto/planck_353_pb/allspec.pkl
writing polar/real_spectra_abscal_pb_fakeauto/planck_217/allspec.pkl
writing polar/real_spectra_abscal_pb_fakeauto/planck_217_planck_353/allspec.pkl
writing polar/real_spectra_abscal_pb_fakeauto/planck_353/allspec.pkl


In [7]:
def dump_dof_form_data(hdf5,exp1,exp2):
    name = make_filename(exp1,exp2)
    foldername = os.path.join('polar/dof/from_data',name)
    os.makedirs(foldername,exist_ok=True)
    spectra = {}
    spectra['bins'] = hdf5['signal_fullcross']['bins'][:].flatten()
    for keys in spectra_keys:
        spectra[keys] = {}
        key = f"real_fullcross/{make_keyname(exp1,exp2)}/{keys}"
        spectra[keys]['nub'] = hdf5[key]['nub'][:]
        spectra[keys]['Cb'] = hdf5[key]['Cb'][:]
    filename = os.path.join(foldername,'allspec.pkl')
    print(f"writing {filename}")
    pk.dump(spectra,open(filename,'wb'))

In [8]:
def dump_dof_calculated(hdf5,exp1,exp2):
    
    
    name = make_filename(exp1,exp2)
    foldername = os.path.join('polar/dof/calculated',name)
    os.makedirs(foldername,exist_ok=True)
    spectra = {}
    spectra['bins'] = hdf5['signal_fullcross']['bins'][:].flatten()
    for keys in spectra_keys:
        spectra[keys] = {}
        #key = f"real_fullcross/{make_keyname(exp1,exp2)}/{keys}"
        
        bias = hdf5[f"noise_fullcross/{make_keyname(exp1,exp2)}/{keys}"]['Cb'][:,:].mean(axis=0)
        std = hdf5[f"noise_fullcross/{make_keyname(exp1,exp2)}/{keys}"]['Cb'][:,:].std(axis=0)
        dof = (bias/std)**2*2
        
        
        spectra[keys]['nub'] = dof
        #spectra[keys]['Cb'] = hdf5[key]['Cb'][:]
    filename = os.path.join(foldername,'allspec.pkl')
    print(f"writing {filename}")
    pk.dump(spectra,open(filename,'wb'))

In [10]:
for i, exp1 in enumerate(freq_name):
    for j, exp2 in enumerate(freq_name):
        if i <= j:
            dump_dof_calculated(hdf5,exp1,exp2)

writing polar/dof/calculated/planck_143/allspec.pkl
writing polar/dof/calculated/planck_143_pb/allspec.pkl
writing polar/dof/calculated/planck_143_planck_217/allspec.pkl
writing polar/dof/calculated/planck_143_planck_353/allspec.pkl
writing polar/dof/calculated/pb/allspec.pkl
writing polar/dof/calculated/planck_217_pb/allspec.pkl
writing polar/dof/calculated/planck_353_pb/allspec.pkl
writing polar/dof/calculated/planck_217/allspec.pkl
writing polar/dof/calculated/planck_217_planck_353/allspec.pkl
writing polar/dof/calculated/planck_353/allspec.pkl


In [9]:
dof = {}
for i, exp1 in enumerate(freq_name):
    for j, exp2 in enumerate(freq_name):
        if i == j:
            dof[make_filename(exp1,exp2)] = {}
            for keys in spectra_keys:
                bias = hdf5[f"noise_fullcross/{make_keyname(exp1,exp2)}/{keys}"]['Cb'][:,:].mean(axis=0)
                std = hdf5[f"noise_fullcross/{make_keyname(exp1,exp2)}/{keys}"]['Cb'][:,:].std(axis=0)
                
                dof[make_filename(exp1,exp2)][keys] = (bias/std)**2*2

In [10]:
dof.keys()

dict_keys(['planck_143', 'pb', 'planck_217', 'planck_353'])

In [14]:
planck ={}
for keys in spectra_keys:
    planck[keys] = {}
    planck_arr = []
    for key in dof.keys():
        if 'planck' in key:
            planck_arr.append(dof[key][keys])
    planck[keys]['nub'] = gmean(np.array(planck_arr))
planck['bins'] = hdf5['signal_fullcross']['bins'][:].flatten()
foldername = os.path.join('polar/dof/calculated','planck')
filename = os.path.join(foldername,'allspec.pkl')
print(f"writing {filename}")
#pk.dump(planck,open(filename,'wb'))

writing polar/dof/calculated/planck/allspec.pkl


In [18]:
planck_pb ={}
for keys in spectra_keys:
    planck_pb[keys] = {}
    planck_arr = []
    for key in dof.keys():
        planck_arr.append(dof[key][keys])
    planck_pb[keys]['nub'] = gmean(np.array(planck_arr))
planck_pb['bins'] = hdf5['signal_fullcross']['bins'][:].flatten()
foldername = os.path.join('polar/dof/calculated','planck_pb')
filename = os.path.join(foldername,'allspec.pkl')
print(f"writing {filename}")
pk.dump(planck_pb,open(filename,'wb'))

writing polar/dof/calculated/planck_pb/allspec.pkl


In [21]:
planck_pb_new = {}
pb_dof = dof['pb']

for keys in spectra_keys:
    planck_pb_new[keys] = {}
    planck_pb_new[keys]['nub'] = np.sqrt(planck[keys]['nub'] * pb_dof[keys])

planck_pb_new['bins'] = hdf5['signal_fullcross']['bins'][:].flatten()
foldername = os.path.join('polar/dof/calculated','planck_pb_final')
os.makedirs(foldername,exist_ok=True)
filename = os.path.join(foldername,'allspec.pkl')
print(f"writing {filename}")
pk.dump(planck_pb_new,open(filename,'wb'))

writing polar/dof/calculated/planck_pb_final/allspec.pkl


In [22]:
planck_pb_new['bb']

{'nub': array([  1.27022008,  55.90259788, 148.10977831, 217.88319117,
        302.80658937, 355.34167729, 413.04791381, 533.6669225 ,
        616.4300294 , 648.42539471, 810.99225452, 792.30354936,
        806.85494276, 503.60722077])}